In [ ]:
%pip list

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

# Load environment variables from .env file
load_dotenv()
cohere_api_key = os.getenv("COHERE_API_KEY")
if not cohere_api_key:
    raise ValueError("COHERE_API_KEY not found in .env file")



In [ ]:
# GET THE YOUTUBE VIDEO, GIVEN THE URL 

In [ ]:
import re
# get video id:
def get_video_id(url):    
    # Regex pattern to match YouTube video URLs
    pattern = r'https:\/\/www\.youtube\.com\/watch\?v=([a-zA-Z0-9_-]{11})'
    match = re.search(pattern, url)
    return match.group(1) if match else None


In [ ]:
youtube_video_url = "https://www.youtube.com/watch?v=tSrN8eRMEAE"

youtube_vid_id = get_video_id(youtube_video_url)
youtube_vid_id

# OUTPUT:
# 'tSrN8eRMEAE'

In [ ]:
# GET THE YOUTUBE TRANSCRIPT

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

Youtube_transcript_key  = YouTubeTranscriptApi()

transcript_list = Youtube_transcript_key.list(youtube_vid_id)
transcript_list
# def get_transcript(url):
#     # Extracts the video ID from the URL
#     video_id = get_video_id(url)

#     # Create a YouTubeTranscriptApi() object
#     ytt_api = YouTubeTranscriptApi()

#     # Fetch the list of available transcripts for the given YouTube video
#     transcripts = ytt_api.list(video_id)

#     transcript = ""
#     for t in transcripts:
#         # Check if the transcript's language is English
#         if t.language_code == 'en':
#             if t.is_generated:
#                 # If no transcript has been set yet, use the auto-generated one
#                 if len(transcript) == 0:
#                     transcript = t.fetch()
#             else:
#                 # If a manually created transcript is found, use it (overrides auto-generated)
#                 transcript = t.fetch()
#                 break  # Prioritize the manually created transcript, exit the loop

#     return transcript if transcript else None

In [ ]:
for transcript in transcript_list:
    print(transcript)
    print("-" * 20)
    print(transcript.is_generated)
    print("-" * 20)
    print(transcript.language_code)
    print("-" * 20)
    print(transcript.is_translatable)
    break


In [ ]:
fetched_transcript = YouTubeTranscriptApi().fetch(youtube_vid_id, languages=['en'])
fetched_transcript

In [ ]:
# Convert to raw Data 

fetched_transcript_in_raw = fetched_transcript.to_raw_data()
fetched_transcript_in_raw

In [ ]:
# NOW CONVERT TO PLAIN TEXT . 

In [ ]:
plain_text = " ".join(chunk["text"] for chunk in fetched_transcript_in_raw)
plain_text

In [ ]:
# TEXT SPLITTING

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
docs = text_splitter.create_documents(texts=[plain_text])

In [ ]:
len(docs)

In [ ]:
# NOW DEFINE LLM TO USE 

from langchain_cohere import ChatCohere

llm = ChatCohere(
    cohere_api_key=cohere_api_key,
    model="command-a-03-2025",  # model selection can be changed
)



In [ ]:
# test llm
print(llm.invoke("What is ai , in one line?"))



In [ ]:
# SET UP EMBEDDING MODEL

In [ ]:
# %pip install sentence-transformers

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
text = "LangChain makes working with LLMs easier."
embedding_vector = embedding.embed_query(text)
print(embedding_vector[30:50])



In [ ]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(docs, embedding)

In [ ]:
vector_store.index_to_docstore_id

In [ ]:
vector_store.get_by_ids

In [ ]:
vector_store.similarity_search("what to learn to avoid ai taking my job",k = 5)

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5},search_type="similarity")
retriever

In [ ]:
retriever.get_relevant_documents("what to learn to avoid ai taking my job")

In [ ]:
# Set up QA template.

In [ ]:
qa_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert assistant providing detailed and accurate answers based on the following video content. Your responses should be:
1. Precise and free from repetition
2. Consistent with the information provided in the video
3. Well-organized and easy to understand
4. Focused on addressing the user's question directly
If you encounter conflicting information in the video content, use your best judgment to provide the most likely correct answer based on context.
Note: In the transcript, "Text" refers to the spoken words in the video, and "start" indicates the timestamp when that part begins in the video.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Relevant Video Context: {context}
Based on the above context, please answer the following question:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [ ]:
# from langchain.prompts import PromptTemplate

# promt_template = PromptTemplate{
#     input_variables=["context", "question"],
#     template=qa_template}

In [ ]:
# qa chain

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": qa_template},
    verbose=True
)

KeyboardInterrupt: 